<a href="https://colab.research.google.com/github/HeMANSC/GenAI/blob/main/Copy_of_YOLOv8final_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## YOLO V8 model
By HEMANT SINGH


## GITHUB LINK


In [ ]:
# Install Ultralytics YOLOv8
!pip install ultralytics

import IPython
from google.colab import output
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import base64
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")  # Using YOLOv8 Nano model

def run(img_str):
    # Decode base64 image
    decimg = base64.b64decode(img_str.split(',')[1], validate=True)
    decimg = Image.open(BytesIO(decimg))
    decimg = np.array(decimg, dtype=np.uint8)
    decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

    # Run YOLOv8 inference
    results = model(decimg)

    # Draw detections
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())
            label = f"{model.names[cls]} {conf:.2f}"
            cv2.rectangle(decimg, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(decimg, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Encode the processed image to base64
    _, encimg = cv2.imencode(".jpg", decimg, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
    img_str = "data:image/jpeg;base64," + base64.b64encode(encimg.tobytes()).decode('utf-8')
    return IPython.display.JSON({'img_str': img_str})

output.register_callback('notebook.run', run)

from IPython.display import display, Javascript
from google.colab.output import eval_js

def use_cam(quality=0.8):
  js = Javascript('''
    async function useCam(quality) {
      const div = document.createElement('div');
      document.body.appendChild(div);
      const video = document.createElement('video');
      video.style.display = 'None';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      display_size = 500;
      const src_canvas = document.createElement('canvas');
      src_canvas.width  = display_size;
      src_canvas.height = display_size * video.videoHeight / video.videoWidth;
      const src_canvasCtx = src_canvas.getContext('2d');
      src_canvasCtx.translate(src_canvas.width, 0);
      src_canvasCtx.scale(-1, 1);
      div.appendChild(src_canvas);

      const dst_canvas = document.createElement('canvas');
      dst_canvas.width  = src_canvas.width;
      dst_canvas.height = src_canvas.height;
      const dst_canvasCtx = dst_canvas.getContext('2d');
      div.appendChild(dst_canvas);

      const btn_div = document.createElement('div');
      document.body.appendChild(btn_div);
      const exit_btn = document.createElement('button');
      exit_btn.textContent = 'Exit';
      var exit_flg = true;
      exit_btn.onclick = function() { exit_flg = false };
      btn_div.appendChild(exit_btn);

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      var send_flg = false;
      async function _canvasUpdate() {
        src_canvasCtx.drawImage(video, 0, 0, video.videoWidth, video.videoHeight, 0, 0, src_canvas.width, src_canvas.height);
        if (send_flg == false) {
            send_flg = true;
            const img = src_canvas.toDataURL('image/jpeg', quality);
            const result = google.colab.kernel.invokeFunction('notebook.run', [img], {});
            result.then(function(value) {
                parse = JSON.parse(JSON.stringify(value))['data'];
                parse = JSON.parse(JSON.stringify(parse))['application/json'];
                parse = JSON.parse(JSON.stringify(parse))['img_str'];
                var image = new Image();
                image.src = parse;
                image.onload = function() { dst_canvasCtx.drawImage(image, 0, 0); };
                send_flg = false;
            });
        }
        if (exit_flg) {
            requestAnimationFrame(_canvasUpdate);
        } else {
            stream.getVideoTracks()[0].stop();
        }
      }
      _canvasUpdate();
    }
  ''')
  display(js)
  eval_js('useCam({})'.format(quality))

use_cam()


<IPython.core.display.Javascript object>


0: 480x640 1 person, 68.2ms
Speed: 25.9ms preprocess, 68.2ms inference, 512.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 13.5ms
Speed: 3.4ms preprocess, 13.5ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 17.9ms
Speed: 14.1ms preprocess, 17.9ms inference, 14.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 cat, 35.3ms
Speed: 4.1ms preprocess, 35.3ms inference, 16.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 cell phone, 11.9ms
Speed: 4.1ms preprocess, 11.9ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 cell phone, 12.0ms
Speed: 4.1ms preprocess, 12.0ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 wine glass, 1 cell phone, 8.9ms
Speed: 3.2ms preprocess, 8.9ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 wine glass, 7.2ms
Speed: 2